In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import copy
import tensorflow as tf
from sklearn.linear_model import LinearRegression


2024-03-25 12:40:25.316007: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 12:40:25.496624: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 12:40:26.185848: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 12:40:27.740792: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
dataset_cols=['bike_count','hour','temp','humidity','wind','visibility','dew_pt_temp','radiation','rain','snow','functional']
df=pd.read_csv('SeoulBikeData.csv').drop(['Date','Seasons','Holiday'], axis=1)

In [5]:
df.columns=dataset_cols
df['functional']=(df['functional']=='Yes').astype(int)
df=df[df['hour']==12]
df=df.drop(['hour'],axis=1)

In [6]:
df.head()

,bike_count,temp,humidity,wind,visibility,dew_pt_temp,radiation,rain,snow,functional
12,449,1.7,23,1.4,2000,-17.2,1.11,0.0,0.0,1
36,479,4.3,41,1.3,1666,-7.8,1.09,0.0,0.0,1
60,333,5.8,85,1.7,349,3.4,0.43,0.0,0.0,1
84,393,-0.3,38,4.8,1823,-12.9,1.11,0.0,0.0,1
108,321,-2.3,25,0.0,1962,-19.7,0.00,0.0,0.0,1


In [ ]:
for label in df.columns[1:]:
    plt.scatter(df[label],df['bike_count'])
    plt.title(label)
    plt.ylabel('Bike count lable')
    plt.xlabel(label)
    plt.show()

In [8]:
df=df.drop(['wind','visibility','functional'], axis=1)

In [9]:
df.head()

,bike_count,temp,humidity,dew_pt_temp,radiation,rain,snow
12,449,1.7,23,-17.2,1.11,0.0,0.0
36,479,4.3,41,-7.8,1.09,0.0,0.0
60,333,5.8,85,3.4,0.43,0.0,0.0
84,393,-0.3,38,-12.9,1.11,0.0,0.0
108,321,-2.3,25,-19.7,0.00,0.0,0.0


In [10]:
#training,testing,validation data
train,valid,test=np.split(df.sample(frac=1),[int(0.6*len(df)),int(0.8*len(df))])

/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [11]:
def get_xy(dataframe, y_label, x_labels=None):
    dataframe=copy.deepcopy(dataframe)
    if not x_labels:
        X=dataframe[[c for c in dataframe.columns if c!=y_label]].values
    else:
        if len(x_labels)==1:
            X=dataframe[x_labels[0]].values.reshape(-1,1)
        else:
            X=dataframe[x_labels].values
    
    y=dataframe[y_label].values.reshape(-1,1)
    data=np.hstack((X,y))

    return data,X,y


            

In [12]:
_,X_train_temp,y_train_temp=get_xy(train,'bike_count',x_labels=['temp'])
_,X_valid_temp,y_train_temp=get_xy(valid,'bike_count',x_labels=['temp'])
_,X_test_temp,y_train_temp=get_xy(test,'bike_count',x_labels=['temp'])

In [13]:
temp_reg=LinearRegression()
temp_reg.fit(X_train_temp, y_train_temp)

ValueError: Found input variables with inconsistent numbers of samples: [219, 73]

In [ ]:
temp_reg.score(X_test_temp, y_train_temp)

In [ ]:
plt.scatter(X_train_temp, y_train_temp, label='Data', color='blue')
x=tf.linespace(-20, 40, 100)
plt.plot(x, temp_reg.predict(np.array(x).reshape(-1,1)), label='Fit', color='red', linewidth=3)
plt.legend()
plt.title("Bikes Vs Temp")
plt.ylabel('Number of bikes')
plt.xlabel('Temp')
plt.show()


In [21]:
#multiple linear regression
train,valid,test=np.split(df.sample(frac=1),[int(0.6*len(df)),int(0.8*len(df))])
_,X_train_all,y_train_all=get_xy(train,'bike_count',x_labels=df.columns[1:])
_,X_valid_all,y_valid_all=get_xy(valid,'bike_count',x_labels=df.columns[1:])
_,X_test_all,y_test_all=get_xy(test,'bike_count',x_labels=df.columns[1:])

/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


ValueError: The truth value of a Index is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
all_reg=LinearRegression()
all_reg.fit(X_test_all, y_train_all)

In [22]:
all_reg.score(X_test_all, y_test_all)

NameError: name 'X_test_all' is not defined

In [ ]:
#regression with neural net